# Testing the Python code for the Microwave Cavity Impedance Model

**it was veryfied that the numeric codes of the Igor script and the Python script (see trmc_network.py) give the same results**


In [1]:
%matplotlib widget

import quickyplot as qp
import physbits
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib as mpl
print('matplotlib version is : {}'.format(mpl.__version__))

import dataimport as di
import pathlib
from trmc_network import S11ghz
from curvefit_ks import curve_fit

matplotlib version is : 3.2.2


## test data from 10.08.2020

**noteworthy observations**   
* The glass substrate only and the coated substrate should be at the same resonance frequency
* there seems to be a large variation in signal depth and also peak position in the trmc experiment. The reason for this is not fully understood yet. Part of it is mechanical reproduceability

In [2]:
import numeric_ks

#dir = pathlib.Path('y:/math/comsol/ref data/2020-03-09 Cavity Tests')
dir = pathlib.Path('u:/friedrich/TRMC/200810_TiO2_Sorbonne_2/')
file1 = 'R_empty cavity_8300-9200_C2_60ms_1s.tdms'
file2 = 'R_quartz_8300-8700_C2_60ms_1s.tdms'
#file3 = 'R_AD-155_2L_800C_8300-8700_C2_60ms_1s.tdms' #sample data
file3 = 'R_AD-155_4L_800C_8300-8700_C2_60ms_1s.tdms'

# data of empty cavity with solid backplate from march
file4 = 'c:/Users/scp/nextcloud/develop/python/misc/trmc-impedance/ref_data/2020-03-09 Cavity Tests/R-C2-empty-CuPlate064.tdms'
d = di.read_tdms(file4)
d_empty2 = d[1].values

d = di.read_tdms(dir/file1)
d_empty = d[0].values
d = di.read_tdms(dir/file2)
d_quartz = d[0].values
d = di.read_tdms(dir/file3)
d_sample = d[0].values

x1 = d_empty[:,0] / 1000
y1 = d_empty[:,1]

x2 = d_quartz[:,0] / 1000
y2 = d_quartz[:,1]

x3 = d_sample[:,0] / 1000
y3 = d_sample[:,1]

x4 = d_empty2[:,0] / 1000
y4 = d_empty2[:,1]


q = qp.quickyplot('',ptype='LINE',label_loc='upper left',xlabel='frequency in GHz',ylabel='R/R0')

q.addplotxy(x1,y1,'empty')
q.addplotxy(x4,y4,'empty2')
q.addplotxy(x2,y2,'quartz')
q.addplotxy(x3,y3,'Sample TiO2')
q.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# fitting the data I (using the empty cavity as a reference)

* first the empty cavity is fitted to find the cavity length, loss factor and iris diameter
* Note that the substrate and layer thicknesses in the model also add to the total thickness of the cavity!
* Follwing the bare substrate and then the TiO2 coated substrate are fitted
* The different position of the resonance peak can be due to different substrate eps or different substrate thickness. Here the eps is fitted although the substrates are/should be actually identical

**The result shows that by using the empty cavity data to extract all the cavity parameters and using those parameters for the cavity loaded with a sample gives only modest fitting results**





In [3]:
#%%timeit -n1 -r1


def s11_func(freq_ghz,d1,d2,d_iris,loss_fac,copper_S,layer_t,layer_epsr,layer_sig,sub_t,sub_epsr,sub_sig):
    # helper function for fitting
    global s11
    s11.d1 = d1 #first distance in mm, distance between sample and cavity end
    s11.d2 = d2 # 'complementary' distance in mm
    s11.d_iris=d_iris # iris diameter in mm
    s11.loss_fac=loss_fac #copper loss adjustment 
    s11.layer_t=layer_t # layer thickness in mm
    s11.layer_epsr=layer_epsr #dieelectric constant layer
    #s11.layer_sig = layer_sig # conductance layer S/m
    s11.layer_sig = abs(layer_sig) # conductance layer S/m
    s11.sub_t=sub_t # substrate thickness in mm
    s11.sub_epsr=sub_epsr # substrate (quartz) epsr  
    s11.sub_sig = abs(sub_sig)
    s11.copper_S = abs(copper_S)
    # return np.array([s11.calc(x) for x in freq_ghz]) # freq_ghz is an array! , non numpy version   
    return s11.calc(freq_ghz) 
    

q = qp.quickyplot('',ptype='LINE',label_loc='upper right',xlabel='frequency in GHz',ylabel='R/R0')

q.addplotxy(x1,y1,'empty')
q.addplotxy(x2,y2,'quartz')
q.addplotxy(x3,y3,'TiO2')

s11 = S11ghz()
#s11.a = 23

c = curve_fit(s11_func)

######## empty cavity #############
c.set('d1',35.825,False)
c.set('d2',11,True)
c.set('d_iris',9.6,False)
c.set('loss_fac',1e-7,False)
c.set('copper_S',5.5e7,True)
c.set('layer_t',0.001,True)
c.set('layer_epsr',1,True)
c.set('layer_sig',0,True)
c.set('sub_t',1,True) # by adding a substrate with eps=1 we account for the proper total cavity length
c.set('sub_epsr',1,True)
c.set('sub_sig',0,True)

c.fit(x1,y1,method='dogbox')

y = c.calc(x1)
dy = (y-y1)**2
print('empty cavity :')
print(f'chisqr = {np.sqrt(dy.sum())}') 
q.addplotxy(x1,y,'fit empty')

for p in c.plist:
    if not p['fixed']:
        print(f"{p['name']} = {p['val']}")
        


######## quartz loaded cavity #############
c.fix('d1')
c.fix('d2')
c.fix('d_iris')
c.fix('loss_fac')
c.set('layer_t',0.001,True)
c.set('layer_epsr',1,True)
c.set('layer_sig',0,True)
c.set('sub_t',1,True)
c.set('sub_epsr',3.6,False)
c.set('sub_sig',0,True)

c.fit(x2,y2,method='lm')

y = c.calc(x2)
dy = (y-y2)**2
print('\ncavity with quartz substrate:')
print(f'chisqr = {np.sqrt(dy.sum())}') 
k=y.argmin()
fmin = x2[k]
s11.layer_sig = 1
kf = s11.kfactor(fmin,rel_change=0.001)
print(f'kfactor = {kf}')

for p in c.plist:
    if not p['fixed']:
        print(f"{p['name']} = {p['val']}")
        
q.addplotxy(x2,y,'fit quartz')



######## sample loaded cavity #############        
c.fix('d1')
c.fix('d2')
c.fix('d_iris')
c.fix('loss_fac')
c.unfix('sub_epsr')

c.set('layer_t',0.001,True)
c.set('layer_epsr',1,True)
c.set('layer_sig',1,False)

c.fit(x3,y3,method='lm')

#c.set('layer_sig',6.66,False)

y = c.calc(x3)
dy = (y-y3)**2
print('\ncavity with TiO2 layer on quartz substrate:')
print(f'chisqr = {np.sqrt(dy.sum())}') 
k = y.argmin()
fmin = x3[k]
s11.layer_sig = 1
kf = s11.kfactor(fmin,rel_change=0.001)
print(f'kfactor = {kf}')

for p in c.plist:
    if not p['fixed']:
        print(f"{p['name']} = {p['val']}")
        
q.addplotxy(x3,y,'fit TiO2')

# c.layer_sig = 10
# y = c.calc(x3)
# q.addplotxy(x3,y,'TiO2 S=0.1')

q.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

empty cavity :
chisqr = 0.17786436597422153
d1 = 35.13363913943076
d_iris = 10.473708916051189
loss_fac = 3.09980532025005e-07

cavity with quartz substrate:
chisqr = 0.6358448077829555
kfactor = -60279.923356426334
sub_epsr = 4.313551815613199

cavity with TiO2 layer on quartz substrate:
chisqr = 0.8339152549660519
kfactor = -52903.52818535912
layer_sig = -4.243865917185213e-06
sub_epsr = 3.910725096468163


# fitting the data II 
## fit also the loss factor and iris diameter

**this gives us an almost perfect fit, but the physical origin of the parameter change is unclear**    
**Note the lower k-factor in this case!**

In [4]:
 
q = qp.quickyplot('',ptype='LINE',label_loc='upper right',xlabel='frequency in GHz',ylabel='R/R0')

q.addplotxy(x1,y1,'empty')
q.addplotxy(x2,y2,'quartz')
q.addplotxy(x3,y3,'TiO2')

s11 = S11ghz()
#s11.a = 23

c = curve_fit(s11_func)

######## empty cavity #############
c.set('d1',35.825,False)
c.set('d2',11,True)
c.set('d_iris',9.6,False)
c.set('loss_fac',1e-7,False)
c.set('copper_S',5.5e7,True)
c.set('layer_t',0.001,True)
c.set('layer_epsr',1,True)
c.set('layer_sig',0,True)
c.set('sub_t',1,True) # by adding a substrate with eps=1 we account for the proper total cavity length
c.set('sub_epsr',1,True)
c.set('sub_sig',0,True)

c.fit(x1,y1,method='dogbox')

y = c.calc(x1)
dy = (y-y1)**2
print('empty cavity :')
print(f'chisqr = {np.sqrt(dy.sum())}') 
q.addplotxy(x1,y,'fit empty')

for p in c.plist:
    if not p['fixed']:
        print(f"{p['name']} = {p['val']}")
        


######## quartz loaded cavity #############
c.fix('d1')
c.fix('d2')
c.unfix('d_iris')
c.unfix('loss_fac')
c.set('layer_t',0.001,True)
c.set('layer_epsr',1,True)
c.set('layer_sig',0,True)
c.set('sub_t',1,True)
c.set('sub_epsr',3.6,False)
c.set('sub_sig',0,True)

c.fit(x2,y2,method='lm')

y = c.calc(x2)
dy = (y-y2)**2
print('\ncavity with quartz substrate:')
print(f'chisqr = {np.sqrt(dy.sum())}') 
k=y.argmin()
fmin = x2[k]
s11.layer_sig = 1
kf = s11.kfactor(fmin,rel_change=0.001)
print(f'kfactor = {kf}')

for p in c.plist:
    if not p['fixed']:
        print(f"{p['name']} = {p['val']}")
        
q.addplotxy(x2,y,'fit quartz')



######## sample loaded cavity #############        
c.fix('d1')
c.fix('d2')
c.unfix('d_iris')
c.unfix('loss_fac')
c.unfix('sub_epsr')

c.set('layer_t',0.001,True)
c.set('layer_epsr',1,True)
c.set('layer_sig',1,False)

c.fit(x3,y3,method='lm')

#c.set('layer_sig',6.66,False)

y = c.calc(x3)
dy = (y-y3)**2
print('\ncavity with TiO2 layer on quartz substrate:')
print(f'chisqr = {np.sqrt(dy.sum())}') 
k = y.argmin()
fmin = x3[k]
s11.layer_sig = 1
kf = s11.kfactor(fmin,rel_change=0.001)
print(f'kfactor = {kf}')

for p in c.plist:
    if not p['fixed']:
        print(f"{p['name']} = {p['val']}")
        
q.addplotxy(x3,y,'fit TiO2')

# c.layer_sig = 10
# y = c.calc(x3)
# q.addplotxy(x3,y,'TiO2 S=0.1')

q.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

empty cavity :
chisqr = 0.17786436597422153
d1 = 35.13363913943076
d_iris = 10.473708916051189
loss_fac = 3.09980532025005e-07

cavity with quartz substrate:
chisqr = 0.12287070752688847
kfactor = -36958.53746293994
d_iris = 10.797122073839475
loss_fac = 2.3957404862235416e-07
sub_epsr = 4.25987329604719

cavity with TiO2 layer on quartz substrate:
chisqr = 0.1647421174789661
kfactor = -37157.30241358824
d_iris = 10.679195430272651
loss_fac = 2.2674771054990416e-07
layer_sig = 4.990365822445449e-06
sub_epsr = 3.870646553065067


# k factor as a function of the loss factor

In [5]:
f = np.arange(8.3,8.6,0.001)

q = qp.quickyplot('Toms formula loss factor influence',ptype='LINE',label_loc='upper right',
                   xlabel='conductance [S]',ylabel='k-factor')


losslist = [5e-8,1e-7,2e-7,3e-7]

for loss in losslist :
    s11.layer_sig = 0
    s11.loss_fac= loss  
    y = s11.calc(f)
    k=y.argmin() #find the min of the resonance peak
    fmin = f[k]      
    t_in_m = (s11.layer_t*1e-3)

    g = np.logspace(-8,-2,500) # conductance
    sig = g / t_in_m # conductivity
    kfac = np.zeros_like(sig)    

    for k,x in enumerate(sig):
        s11.layer_sig = x
        kfac[k] = s11.kfactor(fmin)        

    q.addplotxy(g,kfac,f'loss={loss:.3}')

q.xlog()
q.show()


q = qp.quickyplot('resonance curves',ptype='LINE',label_loc='lower left',
                 xlabel='frequency [GHz]',ylabel='S11')

f = np.arange(8.3,8.7,0.001)


for loss in losslist:
    s11.layer_sig = 0
    s11.loss_fac= loss
    y = s11.calc(f)
    q.addplotxy(f,y,f'loss={loss}')
q.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# k factor as a function of the iris diameter

In [6]:

f = np.arange(8.3,8.6,0.001)

q = qp.quickyplot('Toms formula iris diameter influence',ptype='LINE',label_loc='upper right',
                   xlabel='conductance [S]',ylabel='k-factor')


irislist = [9.0,9.5,10,10.5,11.0]

s11.loss_fac= 2e-7

for dia in irislist :
    s11.layer_sig = 0
    s11.d_iris = dia
    y = s11.calc(f)
    k=y.argmin() #find the min of the resonance peak
    fmin = f[k]      
    t_in_m = (s11.layer_t*1e-3)

    g = np.logspace(-8,-2,500) # conductance
    sig = g / t_in_m # conductivity
    kfac = np.zeros_like(sig)    

    for k,x in enumerate(sig):
        s11.layer_sig = x
        kfac[k] = s11.kfactor(fmin)        

    q.addplotxy(g,kfac,f'dia={dia}mm')

q.xlog()
q.show()


q = qp.quickyplot('resonance curves',ptype='LINE',label_loc='lower left',
                 xlabel='frequency [GHz]',ylabel='S11')

f = np.arange(8.3,8.7,0.001)


for dia in irislist :
    s11.layer_sig = 0
    s11.d_iris = dia    
    y = s11.calc(f)
    q.addplotxy(f,y,f'dia={dia}mm')
q.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Appendix - k factor calculation

## Conversion of detector voltage to rf reflection coefficient dP/P
   
The power is roughly a quadratic function of the RF detector voltage (n~2):  

$$ P=a V^n$$   

In the dark, the detector voltage is Vo. We make a Taylor expansion at V=Vo:  

$$ P(V_{0}+\Delta V) = P(V_{0}) + a n V_{0}^{n-1} \Delta V$$   
$$ \Delta P(V_{0}+\Delta V) = a n V_{0}^{n-1} \Delta V $$
Finally we get:   

$$ \frac{\Delta P(V_{0}+\Delta V)}{P(V_{0})} =  \frac{a n V_{0}^{n-1} \Delta V}{a V_{0}^{n} } = n \frac{\Delta V}{V_{0}}$$


## calculating conductivity $\sigma$ and conductance G from the laser power $P_{L}$   

The semiconductor sample may be homogenously illuminated by lightpulses with a power density $P_{L}$ given in units of J/cm2. The sample fills the cavity with dimensions b = 10.2mm and a = 22.9mm (X-band) and it has a thickness of d. The RF electric field oscillates in the direction of the shorter side (b). If we assume that all photons having a wavelength $\lambda$ are absorbed, a carrier density of $\Delta N$ is created:

$$ \Delta N = \frac{P_{L} \lambda}{h c} \frac{1}{d}$$   

With c=speed of light and h Plancks constant. The increase charge carrier count corresponds to a change in conductivity:

$$ \Delta\sigma = e \Delta N (\mu_{e} + \mu_{h})  $$   

With $\mu_{e} , \mu_{h}$ mobility of electrons and holes and e the elementary charge. To calculate the total conductance change $\Delta G$ the RF E-field is seeing, we need the area A perpendicular to the Field and thickness L of the sample in the direction of E:

$$ \Delta G = \Delta\sigma \frac{A}{L}  $$   

With A = a*d and L=b we get:

$$ \Delta G = \Delta\sigma \frac{a}{b} d = \Delta\sigma \beta d  $$   

With $\beta = a/b$ (=2.245 for X band)

## k - factor    

The sensitivty k-factor is defined by:

$$ k = \frac{\frac{\Delta P}{P}}{\Delta G}   $$   

With the formulas from above we can express this via measureable quantities and with a well defined reference sample with known mobility we can calculate k:

$$
k = \Delta V \frac{n}{V_{0}} \times \frac{h c}{\beta e (\mu_{e} + \mu_{h}) } \times \frac{1}{P_{L} \lambda}
$$

If k is known , we can caclulate the sum of mobilities:

$$
(\mu_{e} + \mu_{h}) = \Delta V \frac{n}{V_{0}} \times \frac{h c}{\beta e } \times \frac{1}{P_{L} \lambda} \times \frac{1}{k}
$$
